In [1]:
import pandas as pd
from sodapy import Socrata
import folium
from folium.features import GeoJson
import math

import secrets

In [97]:
## Initialize the client
client = Socrata("www.data.act.gov.au",
            app_token=secrets.app_token,
            username=secrets.username,
            password=secrets.password)

## You can initialize the client in "severely rate-limited" form with the following
# client = Socrata("www.data.act.gov.au", None)

street_light_dataset = client.get("n9u5-bt96", limit=100000)
all_paths_dataset = client.get("ee7d-h7nm", limit=100000)

# Path Data

In [98]:
all_paths_df = pd.DataFrame.from_records(all_paths_dataset)
bike_paths_df = all_paths_df[all_paths_df['path_type']=='CYCLEPATH']
pedestrian_paths_df = all_paths_df[all_paths_df['path_type']=='FOOTPATH']

street_light_df = pd.DataFrame.from_records(street_light_dataset)
street_light_df['close_to_light'] = False

In [50]:
## coordinates from Canberra wiki
initial_lat = -35.473469
initial_lng = 149.012375

# Load map centred on average coordinates
my_map = folium.Map(location=[initial_lat, initial_lng], zoom_start=10, tiles="cartodbpositron")

## Style definitions
bike_path_style = {'color': '#228B22', 'fillOpacity': 0.5}

for _idx, bike_path_entry in bike_paths_df.iterrows():
    GeoJson(
        data = bike_path_entry['the_geom'],
        name = 'bike_path',
        style_function = lambda x : bike_path_style
    ).add_to(my_map)

for _idx, street_light_entry in street_light_df.iterrows():
    folium.CircleMarker(
        location = [float(street_light_entry['location']['latitude']), float(street_light_entry['location']['longitude'])],
        radius = 0.2,
        color = "yellow", 
        fill = True,
        fill_color = "yellow",
        fill_opacity = 0.7
    ).add_to(my_map)

#my_map

In [9]:
# cutoff = 5 ## arbitrary for now
# light.close_to_bike_path = False

# for path_segment in bike_path_list:
#       for light in lights_list:

#         if !light.close_to_bike_path:
            
#             P1 = path_segment[0] ## this is just a pair list of lat, long
#             P2 = path_segment[1] ## ditto

#             left_distance = math.sqrt((P1.latitude - light.latitude)**2 + (P1.longitude - light.longitude)**2)
#             if left_distance < cutoff:
#               light.close_to_bike_path = True
#               break

#             right_distance = math.sqrt((P2.latitude - light.latitude)**2 + (P2.longitude - light.longitude)**2)
#             if right_distance < cutoff:
#               light.close_to_bike_path = True
#               break

#             perp_distance = abs((P2.latitude - P1.latitude)*(P1.longitude - light.longitude) - (P1.latitude - light.latitude)*(P2.longitude - P1.longitude)) / math.sqrt((P2.latitude - P1.latitude)**2 + (P2.longitude - P1.longitude)**2)
#             if perp_distance < cutoff:
#               light.close_to_bike_path = True
#               break

In [17]:
def minimum_distance_between_point_and_line(lat_line_A, long_line_A, lat_line_B, long_line_B, lat_point_C, long_point_C):
    """
    Given a line defined by two [lat, long] coordinates A[lat_A, long_A], B[lat_B, long_B] in decimal, and a point C defined similarly,
    return the minimum distance between the point and line
    """

    EARTH_RADIUS = 6371*1000 ## in m

    ## Convert lat and long to radians equivalent
    lat_line_A_radians = lat_line_A * math.pi/180
    long_line_A_radians = long_line_A * math.pi/180
    
    lat_line_B_radians = lat_line_B * math.pi/180
    long_line_B_radians = long_line_B * math.pi/180
    
    lat_point_C_radians = lat_point_C * math.pi/180
    long_point_C_radians = long_point_C * math.pi/180

    ## Find the bearing from A to C (C to A?)
    bearingAC = math.atan2(
        math.sin( long_point_C_radians - long_line_A_radians ) * math.cos(lat_point_C_radians),
        math.cos( lat_line_A_radians ) * math.sin( lat_point_C_radians ) - math.sin( lat_line_A_radians ) * math.cos( lat_point_C_radians ) * math.cos( lat_point_C_radians - lat_line_A_radians ) )

    ## Find the bearing from B to C (C to B?)
    bearingAB = math.atan2(
        math.sin( long_line_B_radians - long_line_A_radians ) * math.cos(lat_line_B_radians),
        math.cos( lat_line_A_radians ) * math.sin( lat_line_B_radians ) - math.sin( lat_line_A_radians ) * math.cos( lat_line_B_radians ) * math.cos( lat_line_B_radians - lat_line_A_radians ) )

    bearingAC_degrees = (bearingAC*180/math.pi + 360) % 360
    bearingAB_degrees = (bearingAB*180/math.pi + 360) % 360

    ## Find the distance from A to C using haversine formula

    a = math.sin((lat_point_C_radians - lat_line_A_radians)/2)**2 + math.cos(lat_line_A_radians) * math.cos(lat_point_C_radians) * math.sin((long_point_C_radians - long_line_A_radians)/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    haversine_distance_a_to_c = EARTH_RADIUS * c ## in metres

    ## Find the distance from B to C using haversine formula

    a2 = math.sin((lat_point_C_radians - lat_line_B_radians)/2)**2 + math.cos(lat_line_B_radians) * math.cos(lat_point_C_radians) * math.sin((long_point_C_radians - long_line_B_radians)/2)**2
    c2 = 2 * math.atan2(math.sqrt(a2), math.sqrt(1-a2))
    haversine_distance_b_to_c = EARTH_RADIUS * c2 ## in metres

    ## Find the distance from A to B using haversine formula

    a3 = math.sin((lat_line_B_radians - lat_line_A_radians)/2)**2 + math.cos(lat_line_A_radians) * math.cos(lat_line_B_radians) * math.sin((long_line_B_radians - long_line_A_radians)/2)**2
    c3 = 2 * math.atan2(math.sqrt(a3), math.sqrt(1-a3))
    haversine_distance_a_to_b = EARTH_RADIUS * c3 ## in metres

    ## Find cross-track distance, dxt ( in metres )

    cross_track_distance = math.asin( math.sin( haversine_distance_a_to_c / EARTH_RADIUS ) * math.sin( bearingAC - bearingAB ) ) * EARTH_RADIUS


    ## Find cross-arc distance, dxa ( our desired quantity )

    ## There are three cases to consider, see: https://stackoverflow.com/questions/32771458/distance-from-lat-lng-point-to-minor-arc-segment

    ## Given relative bearing: bearingAC - bearingAB
    ## Case 1: relative_bearing is obtuse, then dxa = distance_a_to_c
    ## Case 2: Relative bearing is acute AND p4 falls on our arc, dxa = dxt
    ## Case 3: Relative bearing is acute AND p4 falls beyond our arc, dxa = distance_b_to_c

    diff = abs(bearingAC - bearingAB)
    if diff > math.pi:
        diff = 2 * math.pi - diff

    ## Is relative bearing obtuse?
    if diff > (math.pi/2):
        dxa = haversine_distance_a_to_c
    else:
        dxt = cross_track_distance

        ## Is point4 beyond the arc?
        distance14 = math.acos( math.cos( haversine_distance_a_to_c ) / math.cos( dxt / EARTH_RADIUS ) ) * EARTH_RADIUS
        if distance14 > haversine_distance_a_to_b:
            dxa = haversine_distance_b_to_c
        else:
            dxa = abs(dxt)

    print("Haversine A-C: ", haversine_distance_a_to_c)
    print("Haversine B-C: ", haversine_distance_b_to_c)
    print("Haversine A-B: ", haversine_distance_a_to_b)
    print("Cross-Arc: ", dxa)

latA = 3.227511
lonA = 101.724318
latB = 3.222895
lonB = 101.719751
latC = 3.224972
lonC = 101.722932


new_latA = 3.227511
new_lonA = 101.724318
new_latB = 3.222895
new_lonB = 101.719751
new_latC = 3.228272
new_lonC = 101.724532

minimum_distance_between_point_and_line(new_latA, new_lonA, new_latB, new_lonB, new_latC, new_lonC)


Haversine A-C:  87.891258430475
Haversine B-C:  799.5040602949088
Haversine A-B:  721.4736569504547
Cross-Arc:  87.891258430475


In [18]:
# cutoff

# for _idx, bike_path_entry in bike_paths_df[:1].iterrows():
#     point_pair_list = bike_path_entry['the_geom']['coordinates'][0]
    
#     for idx, point_pair in enumerate(point_pair_list): ## i.e [149.12482362501234, -35.17695800091904]
#         bike_path_point_latitude = float(point_pair[0])
#         bike_path_point_longitude = float(point_pair[1])

#         for _idx2, street_light_entry in street_light_df.iterrows():
#             light_latitude = float(street_light_entry['location']['latitude'])
#             light_longitude = float(street_light_entry['location']['longitude'])

#             if idx == 0:
#                 print("left dist only")
#                 left_distance = math.sqrt( (bike_path_point_latitude - light_latitude)**2 + (bike_path_point_longitude - light_longitude)**2 )

#                 if left_distance < cutoff:
#                     break

#             elif idx > 0 and idx < len(point_pair_list):
#                 current_point = point_pair
#                 previous_point = point_pair_list[idx-1]

#                 current_point_bike_path_latitude = float(current_point[0])
#                 current_point_bike_path_longitude = float(current_point[1])

#                 previous_point_bike_path_latitude = float(previous_point[0])
#                 previous_point_bike_path_longitude = float(previous_point[1])

#                 print(f"do left on current_point and perp with line defined by {current_point}, idx = {idx} and with {previous_point}, idx = {idx-1}")

#                 left_distance = math.sqrt( (current_point_bike_path_latitude - light_latitude)**2 + (current_point_bike_path_longitude - light_longitude)**2 )



#             else: ## idx == len(...)
#                 print("we messed up")


# ## In the first case, the path is made up of 6 pairs of points (therefore, 5 segments).
# ## if idx = 0, we're at the start of a pair list and only need to do left_distance and perp_dist with the n+1th point

In [ ]:
# for _idx, bike_path_entry in bike_paths_df[:1].iterrows():
#     point_pair_list = bike_path_entry['the_geom']['coordinates'][0]
#     for idx, point_pair in enumerate(point_pair_list): ## i.e [149.12482362501234, -35.17695800091904]
#         if idx == 0:
#             print("left dist only")
#         elif idx > 0 and idx < len(point_pair_list):
#             current_point = point_pair
#             previous_point = point_pair_list[idx-1]
#             print(f"do left on current_point and perp with line defined by {current_point}, idx = {idx} and with {previous_point}, idx = {idx-1}")
#         else: ## idx == len(...)
#             print("we fucked up")
